In [1]:
# Dependencies.
# Data Cleaning/Engineering.
import pandas as pd
import numpy as np

# Data Scaling/Splitting.
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# NN Model.
import tensorflow as tf
import keras_tuner as kt

# Visualization.
import matplotlib.pyplot as plt
import hvplot.pandas
import seaborn as sns
from bokeh.resources import INLINE

In [2]:
# Read in data.
raw_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
raw_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Check out the data.
display(raw_df.info())
display(raw_df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   EIN                     34299 non-null  int64 
 1   NAME                    34299 non-null  object
 2   APPLICATION_TYPE        34299 non-null  object
 3   AFFILIATION             34299 non-null  object
 4   CLASSIFICATION          34299 non-null  object
 5   USE_CASE                34299 non-null  object
 6   ORGANIZATION            34299 non-null  object
 7   STATUS                  34299 non-null  int64 
 8   INCOME_AMT              34299 non-null  object
 9   SPECIAL_CONSIDERATIONS  34299 non-null  object
 10  ASK_AMT                 34299 non-null  int64 
 11  IS_SUCCESSFUL           34299 non-null  int64 
dtypes: int64(4), object(8)
memory usage: 3.1+ MB


None

,EIN,STATUS,ASK_AMT,IS_SUCCESSFUL
count,3.429900e+04,34299.000000,3.429900e+04,34299.000000
mean,5.191852e+08,0.999854,2.769199e+06,0.532406
std,2.451472e+08,0.012073,8.713045e+07,0.498956
min,1.052060e+07,0.000000,5.000000e+03,0.000000
25%,2.748482e+08,1.000000,5.000000e+03,0.000000
50%,4.656317e+08,1.000000,5.000000e+03,1.000000
75%,7.526117e+08,1.000000,7.742000e+03,1.000000
max,9.960869e+08,1.000000,8.597806e+09,1.000000


In [4]:
# Drop the EIN and Name columns.
df = raw_df.drop(columns = ['EIN', 'NAME'])
df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [5]:
# Find unique values among all columns.
df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [6]:
# Determine number of data points for columns containing more than 10 unique values.
df.APPLICATION_TYPE.value_counts()

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [7]:
# Create a cutoff for APPLICATION_TYPE and submit points under the cutoff as Other.
cutoff = ['T3', 'T4', 'T6', 'T5', 'T19'] # Note: could have made this 'automatic' but such a small amt that easier to just hard code.
df['APPLICATION_TYPE'] = df.APPLICATION_TYPE.map(lambda x: x if x in cutoff else "Other")
df.APPLICATION_TYPE.value_counts()

APPLICATION_TYPE
T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: count, dtype: int64

In [8]:
# Look at the next column.
display(df.CLASSIFICATION.value_counts())

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [9]:
# Create a cutoff and assign all values below it to Other.
cutoff = ['C1000', 'C2000', 'C1200', 'C3000', 'C2100'] # Note: could have made this 'automatic' but such a small amt that easier to just hard code.
df['CLASSIFICATION'] = df.CLASSIFICATION.map(lambda x: x if x in cutoff else "Other")
df.CLASSIFICATION.value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [10]:
# Convert categories to numerical data with get_dummies.
df_sub = pd.get_dummies(
    df,
    prefix = ['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS'],
    columns = ['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS'],
    dtype = 'int64'
)

df_sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 41 columns):
 #   Column                        Non-Null Count  Dtype
---  ------                        --------------  -----
 0   STATUS                        34299 non-null  int64
 1   ASK_AMT                       34299 non-null  int64
 2   IS_SUCCESSFUL                 34299 non-null  int64
 3   APPLICATION_TYPE_Other        34299 non-null  int64
 4   APPLICATION_TYPE_T19          34299 non-null  int64
 5   APPLICATION_TYPE_T3           34299 non-null  int64
 6   APPLICATION_TYPE_T4           34299 non-null  int64
 7   APPLICATION_TYPE_T5           34299 non-null  int64
 8   APPLICATION_TYPE_T6           34299 non-null  int64
 9   AFFILIATION_CompanySponsored  34299 non-null  int64
 10  AFFILIATION_Family/Parent     34299 non-null  int64
 11  AFFILIATION_Independent       34299 non-null  int64
 12  AFFILIATION_National          34299 non-null  int64
 13  AFFILIATION_Other             3

In [11]:
# Isolate features and target.
features = df_sub.drop(columns = ['IS_SUCCESSFUL'])
target = df_sub.loc[:, 'IS_SUCCESSFUL']

In [12]:
# Create X and y variables.
X = features
y = target

display(X.head())
display(y.head())

,STATUS,ASK_AMT,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,0,0,1,0,0,0,1,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


0    1
1    1
2    0
3    1
4    1
Name: IS_SUCCESSFUL, dtype: int64

In [13]:
# Create a train_test_split.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 1, stratify = y)
print(X_train.shape, X_test.shape)

(25724, 40) (8575, 40)


In [14]:
# Scale the training and testing features.
scaler = StandardScaler()

scaler.fit(X_train)
X_train_sc = scaler.transform(X_train)
X_test_sc = scaler.transform(X_test)

# Model 1

In [15]:
# Now, going to create the first NN model.
nn_model_1 = tf.keras.models.Sequential()

# First layer.
nn_model_1.add(tf.keras.layers.Dense(units = 7, activation = 'relu', input_dim = len(X.columns)))

# Output layer (uses a Sigmoid activation for probability).
nn_model_1.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Summarize this.
nn_model_1.summary()

C:\Users\emteb\anaconda3\envs\dev\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 7)                   │             287 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │               8 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 295 (1.15 KB)

 Trainable params: 295 (1.15 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
# Make custom callback to save every five epochs.
class MyModelCheckpoint(tf.keras.callbacks.ModelCheckpoint):
    def on_epoch_end(self, epoch, logs = None):
        if (epoch + 1) % 5 == 0:
            super().on_epoch_end(epoch, logs)

custom_callback = MyModelCheckpoint(
    filepath = 'model_checkpoints/nn_1_weights_{epoch:02d}.weights.h5',
    save_weights_only = True,
    save_freq = 'epoch',
)

In [17]:
# Compile and fit model.
nn_model_1.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

# Fit that model!
fit_1 = nn_model_1.fit(
    X_train_sc,
    y_train,
    epochs = 100,
    callbacks = [custom_callback]
)

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 864us/step - accuracy: 0.6204 - loss: 0.6849
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 844us/step - accuracy: 0.7218 - loss: 0.5756
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 843us/step - accuracy: 0.7256 - loss: 0.5641
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 850us/step - accuracy: 0.7227 - loss: 0.5665
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 865us/step - accuracy: 0.7307 - loss: 0.5585
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 846us/step - accuracy: 0.7226 - loss: 0.5666
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 842us/step - accuracy: 0.7285 - loss: 0.5596
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 834us/step - accuracy: 0.7229 - loss: 0.5638
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 845us/step - accuracy: 0.7270 - loss: 0.5591
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 857us/step - accuracy: 0.7306 - loss: 0.5571
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 840us/step - accuracy: 0.7309 - loss: 0.5557
Epoch 12/100
804/80

In [18]:
# Plot loss and accuracy over time.
v_df = pd.DataFrame(fit_1.history)
v_df.index += 1

plot = v_df.hvplot.line(y = 'loss', color = 'firebrick') + v_df.hvplot.line(y = 'accuracy', color = 'cornflowerblue')

hvplot.save(plot, "images/model_1_metrics.png")

plot

:Layout
   .Curve.I  :Curve   [index]   (loss)
   .Curve.II :Curve   [index]   (accuracy)

In [19]:
# Evaluate the model.
model_1_loss, model_1_acc = nn_model_1.evaluate(X_test_sc, y_test, verbose = 2)
print(
f"""
Model 1 Metrics
--------------------------------------------------
Loss: {model_1_loss}
Accuracy: {model_1_acc}
"""
)

268/268 - 0s - 1ms/step - accuracy: 0.7258 - loss: 0.5580

Model 1 Metrics
--------------------------------------------------
Loss: 0.5580150485038757
Accuracy: 0.7258309125900269



In [20]:
# That's horrible. Let's do that again.

# Model 2

This time. I want to change the amount of layers in the NN model to see if I can improve upon the performance.  
I will try a layer of 5 neurons followed by a layer of 3 neurons followed by the output layer.

In [21]:
# Now, going to create the NN model.
nn_model_2 = tf.keras.models.Sequential()

# First layer.
nn_model_2.add(tf.keras.layers.Dense(units = 5, activation = 'relu', input_dim = len(X.columns)))

# Second Layer.
nn_model_2.add(tf.keras.layers.Dense(units = 3, activation = 'relu'))

# Output layer (uses a Sigmoid activation for probability).
nn_model_2.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Summarize this.
nn_model_2.summary()

C:\Users\emteb\anaconda3\envs\dev\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                      │ (None, 5)                   │             205 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 3)                   │              18 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 1)                   │               4 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 227 (908.00 B)

 Trainable params: 227 (908.00 B)

 Non-trainable params: 0 (0.00 B)

In [22]:
# Compile and fit model.
nn_model_2.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

# Custom callback.
custom_callback = MyModelCheckpoint(
    filepath = 'model_checkpoints/nn_2_weights_{epoch:02d}.weights.h5',
    save_weights_only = True,
    save_freq = 'epoch',
)

# Fit that model!
fit_2 = nn_model_2.fit(
    X_train_sc,
    y_train,
    epochs = 100,
    callbacks = [custom_callback]
)

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 921us/step - accuracy: 0.6265 - loss: 0.6616
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 931us/step - accuracy: 0.7150 - loss: 0.5834
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 920us/step - accuracy: 0.7146 - loss: 0.5733
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 923us/step - accuracy: 0.7128 - loss: 0.5750
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 943us/step - accuracy: 0.7151 - loss: 0.5686
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 925us/step - accuracy: 0.7279 - loss: 0.5648
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 920us/step - accuracy: 0.7264 - loss: 0.5627
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 922us/step - accuracy: 0.7259 - loss: 0.5639
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 917us/step - accuracy: 0.7263 - loss: 0.5646
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 946us/step - accuracy: 0.7277 - loss: 0.5576
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 910us/step - accuracy: 0.7281 - loss: 0.5570
Epoch 12/100
804/80

In [23]:
# Plot loss and accuracy over time.
v_df = pd.DataFrame(fit_2.history)
v_df.index += 1

plot = v_df.hvplot.line(y = 'loss', color = 'firebrick') + v_df.hvplot.line(y = 'accuracy', color = 'cornflowerblue')

hvplot.save(plot, "images/model_2_metrics.png")

plot

:Layout
   .Curve.I  :Curve   [index]   (loss)
   .Curve.II :Curve   [index]   (accuracy)

In [24]:
# Evaluate the model.
model_2_loss, model_2_acc = nn_model_2.evaluate(X_test_sc, y_test, verbose = 2)
print(
f"""
Model 1 Metrics
--------------------------------------------------
Loss: {model_2_loss}
Accuracy: {model_2_acc}
"""
)

268/268 - 0s - 1ms/step - accuracy: 0.7290 - loss: 0.5560

Model 1 Metrics
--------------------------------------------------
Loss: 0.5559841394424438
Accuracy: 0.7289795875549316



In [25]:
# Again, horrible.

# Model 3

I don't think I should be just throwing more and more neurons at this problem until it gets to 75% or higher accuracy...  
So, I am going to mess with the activation function for the hidden layers now and see if performance improves because of it.  
I will be switching from relu to tanh to see if it will improve the model.

In [26]:
# Now, going to create the NN model.
nn_model_3 = tf.keras.models.Sequential()

# First layer.
nn_model_3.add(tf.keras.layers.Dense(units = 5, activation = 'tanh', input_dim = len(X.columns)))

# Second Layer.
nn_model_3.add(tf.keras.layers.Dense(units = 3, activation = 'tanh'))

# Output layer (uses a Sigmoid activation for probability).
nn_model_3.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Summarize this.
nn_model_3.summary()

C:\Users\emteb\anaconda3\envs\dev\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_5 (Dense)                      │ (None, 5)                   │             205 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 3)                   │              18 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 1)                   │               4 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 227 (908.00 B)

 Trainable params: 227 (908.00 B)

 Non-trainable params: 0 (0.00 B)

In [27]:
# Compile and fit model.
nn_model_3.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

# Custom callback.
custom_callback = MyModelCheckpoint(
    filepath = 'model_checkpoints/nn_3_weights_{epoch:02d}.weights.h5',
    save_weights_only = True,
    save_freq = 'epoch',
)

# Fit that model!
fit_3 = nn_model_3.fit(
    X_train_sc,
    y_train,
    epochs = 100,
    callbacks = [custom_callback]
)

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 928us/step - accuracy: 0.6620 - loss: 0.6322
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 933us/step - accuracy: 0.7295 - loss: 0.5690
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 919us/step - accuracy: 0.7236 - loss: 0.5708
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 933us/step - accuracy: 0.7304 - loss: 0.5616
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 948us/step - accuracy: 0.7307 - loss: 0.5601
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 921us/step - accuracy: 0.7298 - loss: 0.5613
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 929us/step - accuracy: 0.7334 - loss: 0.5564
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 917us/step - accuracy: 0.7311 - loss: 0.5561
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 906us/step - accuracy: 0.7312 - loss: 0.5565
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 943us/step - accuracy: 0.7266 - loss: 0.5596
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 923us/step - accuracy: 0.7293 - loss: 0.5549
Epoch 12/100
804/80

In [28]:
# Plot loss and accuracy over time.
v_df = pd.DataFrame(fit_3.history)
v_df.index += 1

plot = v_df.hvplot.line(y = 'loss', color = 'firebrick') + v_df.hvplot.line(y = 'accuracy', color = 'cornflowerblue')

hvplot.save(plot, "images/model_3_metrics.png")

plot

:Layout
   .Curve.I  :Curve   [index]   (loss)
   .Curve.II :Curve   [index]   (accuracy)

In [29]:
# Evaluate the model.
model_3_loss, model_3_acc = nn_model_3.evaluate(X_test_sc, y_test, verbose = 2)
print(
f"""
Model 1 Metrics
--------------------------------------------------
Loss: {model_3_loss}
Accuracy: {model_3_acc}
"""
)

268/268 - 0s - 1ms/step - accuracy: 0.7306 - loss: 0.5599

Model 1 Metrics
--------------------------------------------------
Loss: 0.5598627328872681
Accuracy: 0.7306122183799744



# Model 4

Still not hitting the target so I think it is time to add a large amount of neurons.

In [30]:
# Now, going to create the NN model.
nn_model_4 = tf.keras.models.Sequential()

# First layer.
nn_model_4.add(tf.keras.layers.Dense(units = 100, activation = 'tanh', input_dim = len(X.columns)))

# Second Layer.
nn_model_4.add(tf.keras.layers.Dense(units = 100, activation = 'tanh'))

# Third Layer.
nn_model_4.add(tf.keras.layers.Dense(units = 100, activation = 'tanh'))

# Output layer (uses a Sigmoid activation for probability).
nn_model_4.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Summarize this.
nn_model_4.summary()

C:\Users\emteb\anaconda3\envs\dev\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_8 (Dense)                      │ (None, 100)                 │           4,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 100)                 │          10,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 100)                 │          10,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 1)                   │             101 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 24,401 (95.32 KB)

 Trainable params: 24,401 (95.32 KB)

 Non-trainable params: 0 (0.00 B)

In [31]:
# Compile and fit model.
nn_model_4.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

# Custom callback.
custom_callback = MyModelCheckpoint(
    filepath = 'model_checkpoints/nn_4_weights_{epoch:02d}.weights.h5',
    save_weights_only = True,
    save_freq = 'epoch',
)

# Fit that model!
fit_4 = nn_model_4.fit(
    X_train_sc,
    y_train,
    epochs = 100,
    callbacks = [custom_callback]
)

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7109 - loss: 0.5880
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7297 - loss: 0.5625
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7270 - loss: 0.5621
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7312 - loss: 0.5546
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7302 - loss: 0.5553
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7313 - loss: 0.5521
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7338 - loss: 0.5488
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7349 - loss: 0.5466
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7326 - loss: 0.5507
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7364 - loss: 0.5453
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7371 - loss: 0.5436
Epoch 12/100
804/804 ━━━━━━━━━━━━━━━━━━━━

In [32]:
# Plot loss and accuracy over time.
v_df = pd.DataFrame(fit_4.history)
v_df.index += 1

plot = v_df.hvplot.line(y = 'loss', color = 'firebrick') + v_df.hvplot.line(y = 'accuracy', color = 'cornflowerblue')

hvplot.save(plot, "images/model_4_metrics.png")

plot

:Layout
   .Curve.I  :Curve   [index]   (loss)
   .Curve.II :Curve   [index]   (accuracy)

In [33]:
# Evaluate the model.
model_4_loss, model_4_acc = nn_model_4.evaluate(X_test_sc , y_test, verbose = 2)
print(
f"""
Model 1 Metrics
--------------------------------------------------
Loss: {model_4_loss}
Accuracy: {model_4_acc}
"""
)

268/268 - 0s - 1ms/step - accuracy: 0.7292 - loss: 0.5660

Model 1 Metrics
--------------------------------------------------
Loss: 0.5659773945808411
Accuracy: 0.7292128205299377



# Model 5

I seem to be falling short of making this model worthwhile.  
I will now use the hyperparameter tuner to look for an ideal model.  


In [34]:
# Function to use for hyperparameter tuner.
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow the tuner to choose which activation function to use.
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])

    # Allow the tuner to decide the number of nodes in the first layer.
    nn_model.add(
        tf.keras.layers.Dense(
            units = hp.Int(
                'first_units',
                min_value = 1,
                max_value = 10,
                step = 2),
            activation = activation,
            input_dim = len(X.columns)))

    # Allow the tuner to flesh out the rest of the model at will.
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))

    # Make the output layer.
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_model

In [35]:
tuner = kt.Hyperband(
    create_model,
    objective = 'val_accuracy',
    max_epochs = 25,
    hyperband_iterations = 2
)

Reloading Tuner from .\untitled_project\tuner0.json


In [36]:
tuner.search(
    X_train_sc,
    y_train,
    epochs = 25,
    validation_data = (X_test_sc, y_test)
)

In [37]:
# Get the best model from the tuner.
best_param = tuner.get_best_hyperparameters(1)[0]
print(best_param.values)

# Evaluate the best model against the test data.
nn_model_5 = tuner.get_best_models(1)[0]
model_5_loss, model_5_acc = nn_model_5.evaluate(X_test_sc, y_test, verbose = 2)
print(
f"""
Model 1 Metrics
--------------------------------------------------
Loss: {model_5_loss}
Accuracy: {model_5_acc}
"""
)

# Custom callback.
custom_callback = MyModelCheckpoint(
    filepath = 'model_checkpoints/nn_5_weights_{epoch:02d}.weights.h5',
    save_weights_only = True,
    save_freq = 'epoch',
)

# Fit that model!
fit_5 = nn_model_5.fit(
    X_train_sc,
    y_train,
    epochs = 100,
    callbacks = [custom_callback]
)

{'activation': 'sigmoid', 'first_units': 7, 'num_layers': 2, 'units_0': 7, 'units_1': 3, 'units_2': 7, 'units_3': 3, 'units_4': 1, 'units_5': 7, 'tuner/epochs': 25, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}



C:\Users\emteb\anaconda3\envs\dev\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\emteb\anaconda3\envs\dev\lib\site-packages\keras\src\saving\saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


268/268 - 1s - 2ms/step - accuracy: 0.7313 - loss: 0.5753

Model 1 Metrics
--------------------------------------------------
Loss: 0.5753284692764282
Accuracy: 0.7313119769096375

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 967us/step - accuracy: 0.7320 - loss: 0.5734
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 978us/step - accuracy: 0.7272 - loss: 0.5762
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 964us/step - accuracy: 0.7311 - loss: 0.5737
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 957us/step - accuracy: 0.7244 - loss: 0.5793
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7284 - loss: 0.5757  
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 977us/step - accuracy: 0.7343 - loss: 0.5712
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 978us/step - accuracy: 0.7324 - loss: 0.5702
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 979us/step - accuracy: 0.7335 - loss: 0.5695
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 969us/step - accuracy: 0.7350 - loss: 0.5664
Epoch 10/100
804/8

In [38]:
# Plot loss and accuracy over time.
v_df = pd.DataFrame(fit_5.history)
v_df.index += 1

plot = v_df.hvplot.line(y = 'loss', color = 'firebrick') + v_df.hvplot.line(y = 'accuracy', color = 'cornflowerblue')

hvplot.save(plot, "images/model_5_metrics.png")

plot

:Layout
   .Curve.I  :Curve   [index]   (loss)
   .Curve.II :Curve   [index]   (accuracy)

In [39]:
# Metrics for updated model 5.
model_5_loss, model_5_acc = nn_model_5.evaluate(X_test_sc, y_test, verbose = 2)
print(
f"""
Model 1 Metrics
--------------------------------------------------
Loss: {model_5_loss}
Accuracy: {model_5_acc}
"""
)

268/268 - 0s - 770us/step - accuracy: 0.7284 - loss: 0.5569

Model 1 Metrics
--------------------------------------------------
Loss: 0.5568950176239014
Accuracy: 0.728396475315094



In [40]:
# We didn't quite get to 75% accuracy within 4 model iterations. However, using the hyperparameter tuner gave us the best performance.
# What more can be done? ...
# To improve performance, it could be best to manipulate the input data 'better.'
# ... or at least play with the input data enough to eek out the best performance possible from the model.

In [41]:
# Save the models. Note: got to start using .keras for the extensions as hdf5 is going to be deprecated.
nn_model_1.save("models/nn_model_1.hdf5")
nn_model_2.save("models/nn_model_2.hdf5")
nn_model_3.save("models/nn_model_3.hdf5")
nn_model_4.save("models/nn_model_4.hdf5")
nn_model_5.save("models/AIphabetSoupCharity_Optimization.hdf5")